<a href="https://colab.research.google.com/github/Arup3201/Summarization-Project-using-Pointer-Gen/blob/main/Get_To_The_Point_Summarization_with_Pointer_Generator_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pathlib

In [2]:
path_to_cnn_stories = tf.keras.utils.get_file(
    origin="https://huggingface.co/datasets/cnn_dailymail/resolve/main/data/cnn_stories.tgz",
    extract=True
)

path_to_dailymail_stories = tf.keras.utils.get_file(
    origin="https://huggingface.co/datasets/cnn_dailymail/resolve/main/data/dailymail_stories.tgz",
    extract=True
)

375893739/375893739 [==============================] - 4s 0us/step


In [3]:
path_to_cnn_stories, path_to_dailymail_stories

('/root/.keras/datasets/cnn_stories.tgz',
 '/root/.keras/datasets/dailymail_stories.tgz')

In [4]:
!ls -l /root/.keras/datasets

total 521960
drwxr-xr-x 3 root root      4096 Jun 29 05:51 cnn
-rw-r--r-- 1 root root 158577824 Jun 29 05:51 cnn_stories.tgz
drwxr-xr-x 3 root root      4096 Jun 29 05:51 dailymail
-rw-r--r-- 1 root root 375893739 Jun 29 05:51 dailymail_stories.tgz


In [5]:
cnn_stories_dir = pathlib.Path('/root/.keras/datasets/cnn/stories')
dailymail_stories_dir = pathlib.Path('/root/.keras/datasets/dailymail/stories')

In [6]:
cnn_stories_dir, dailymail_stories_dir

(PosixPath('/root/.keras/datasets/cnn/stories'),
 PosixPath('/root/.keras/datasets/dailymail/stories'))

In [7]:
for story in cnn_stories_dir.glob('*.story'):
  print(story.name)
  break

b542255aba4cf869cf513e08a84b429b8bc0abf2.story


In [8]:
cnn_stories = tf.data.Dataset.list_files(str(cnn_stories_dir/'*.story'))

for f in cnn_stories.take(5):
  print(f.numpy())

b'/root/.keras/datasets/cnn/stories/66463e8b1df792a6965ec21df398305e0442d38a.story'
b'/root/.keras/datasets/cnn/stories/2b87bbacf5f4eb1bce5bf557152028f69fbdf389.story'
b'/root/.keras/datasets/cnn/stories/6a779dc46af68abe061cc11b126d226b5d7bcddc.story'
b'/root/.keras/datasets/cnn/stories/9814d293011b5b225a587e4fe681aa8682d643cb.story'
b'/root/.keras/datasets/cnn/stories/4c9b8420b4581428513e9404e09db12d98279f90.story'


In [9]:
dailymail_stories = tf.data.Dataset.list_files(str(dailymail_stories_dir/'*.story'))

for f in dailymail_stories.take(5):
  print(f.numpy())

b'/root/.keras/datasets/dailymail/stories/ba7fe6f45906be5f33f2b3ce10db1bbc49e07554.story'
b'/root/.keras/datasets/dailymail/stories/092de3cb30f7b237aa3a3138d48fa4bd47ce9f43.story'
b'/root/.keras/datasets/dailymail/stories/abe3b004fdfbb069dc740cf8c5bb0f589bc83294.story'
b'/root/.keras/datasets/dailymail/stories/b56dc9831146a4aed7bd50883d47faf574eb3279.story'
b'/root/.keras/datasets/dailymail/stories/b3b933f728c7c3151c112ebef7360cfe40c31a84.story'


In [10]:
file_path = '/root/.keras/datasets/cnn/stories/0b920d4cf93a3e39bc885630426b62f0e3e4bc41.story'
tf.io.read_file(file_path)

<tf.Tensor: shape=(), dtype=string, numpy=b'(CNN) -- A Florida man is accused of strangling a family puppy, chopping it into pieces and cooking its ribs on the stove, Tampa police said.\n\nThomas Elliot Huggins, 25, was arrested Thursday after a family member called police.\n\n"When officers arrived, they found the dog\'s ribs cooked in a pot on the stove," Tampa police said. "The dog\'s head was in the garbage."\n\nHuggins had strangled the 5- to 6-month-old puppy and chopped it into quarters, storing pieces in the freezer for future meals, authorities said.\n\nHe was arrested on suspicion of animal cruelty. Animal control officials took the remains of the puppy.\n\nSmugglers drive Thailand\'s grim trade in dog meat\n\nA national spotlight on animal abuse\n\n@highlight\n\nPolice say Thomas Elliot Huggins strangled and cut up a puppy\n\n@highlight\n\nWhen police arrived, they found the dog\'s ribs cooking on the stove'>

In [24]:
sample_story = tf.io.read_file(file_path)
tf.strings.split(sample_story, sep='\n\n')

<tf.Tensor: shape=(11,), dtype=string, numpy=
array([b'(CNN) -- A Florida man is accused of strangling a family puppy, chopping it into pieces and cooking its ribs on the stove, Tampa police said.',
       b'Thomas Elliot Huggins, 25, was arrested Thursday after a family member called police.',
       b'"When officers arrived, they found the dog\'s ribs cooked in a pot on the stove," Tampa police said. "The dog\'s head was in the garbage."',
       b'Huggins had strangled the 5- to 6-month-old puppy and chopped it into quarters, storing pieces in the freezer for future meals, authorities said.',
       b'He was arrested on suspicion of animal cruelty. Animal control officials took the remains of the puppy.',
       b"Smugglers drive Thailand's grim trade in dog meat",
       b'A national spotlight on animal abuse', b'@highlight',
       b'Police say Thomas Elliot Huggins strangled and cut up a puppy',
       b'@highlight',
       b"When police arrived, they found the dog's ribs cooking

In [25]:
sample_story_sents = tf.strings.split(sample_story, sep='\n\n')
tf.where(sample_story_sents=='@highlight')

<tf.Tensor: shape=(2, 1), dtype=int64, numpy=
array([[7],
       [9]])>

In [26]:
pos = tf.where(sample_story_sents=='@highlight')
tf.reshape(pos, [-1])

<tf.Tensor: shape=(2,), dtype=int64, numpy=array([7, 9])>

In [27]:
pos = tf.reshape(pos, [-1])

<tf.Tensor: shape=(2,), dtype=int64, numpy=array([ 8, 10])>

In [28]:
sample_story_sents[:pos[0]]

<tf.Tensor: shape=(7,), dtype=string, numpy=
array([b'(CNN) -- A Florida man is accused of strangling a family puppy, chopping it into pieces and cooking its ribs on the stove, Tampa police said.',
       b'Thomas Elliot Huggins, 25, was arrested Thursday after a family member called police.',
       b'"When officers arrived, they found the dog\'s ribs cooked in a pot on the stove," Tampa police said. "The dog\'s head was in the garbage."',
       b'Huggins had strangled the 5- to 6-month-old puppy and chopped it into quarters, storing pieces in the freezer for future meals, authorities said.',
       b'He was arrested on suspicion of animal cruelty. Animal control officials took the remains of the puppy.',
       b"Smugglers drive Thailand's grim trade in dog meat",
       b'A national spotlight on animal abuse'], dtype=object)>

In [47]:
sample_article = tf.strings.join(sample_story_sents[:pos[0]], ' ')
sample_article

<tf.Tensor: shape=(), dtype=string, numpy=b'(CNN) -- A Florida man is accused of strangling a family puppy, chopping it into pieces and cooking its ribs on the stove, Tampa police said. Thomas Elliot Huggins, 25, was arrested Thursday after a family member called police. "When officers arrived, they found the dog\'s ribs cooked in a pot on the stove," Tampa police said. "The dog\'s head was in the garbage." Huggins had strangled the 5- to 6-month-old puppy and chopped it into quarters, storing pieces in the freezer for future meals, authorities said. He was arrested on suspicion of animal cruelty. Animal control officials took the remains of the puppy. Smugglers drive Thailand\'s grim trade in dog meat A national spotlight on animal abuse'>

In [30]:
pos+1

<tf.Tensor: shape=(2,), dtype=int64, numpy=array([ 8, 10])>

In [31]:
ind = pos + 1
tf.gather(sample_story_sents, ind)

<tf.Tensor: shape=(2,), dtype=string, numpy=
array([b'Police say Thomas Elliot Huggins strangled and cut up a puppy',
       b"When police arrived, they found the dog's ribs cooking on the stove"],
      dtype=object)>

In [39]:
highlights = tf.gather(sample_story_sents, ind)
highlights + '.'

<tf.Tensor: shape=(2,), dtype=string, numpy=
array([b'Police say Thomas Elliot Huggins strangled and cut up a puppy.',
       b"When police arrived, they found the dog's ribs cooking on the stove."],
      dtype=object)>

In [48]:
sample_summary = tf.strings.join(highlights + '.', ' ')
sample_summary

<tf.Tensor: shape=(), dtype=string, numpy=b"Police say Thomas Elliot Huggins strangled and cut up a puppy. When police arrived, they found the dog's ribs cooking on the stove.">

In [43]:
another_file_path = '/root/.keras/datasets/dailymail/stories/ba7fe6f45906be5f33f2b3ce10db1bbc49e07554.story'
tf.io.read_file(another_file_path)

<tf.Tensor: shape=(), dtype=string, numpy=b"By\nRosemarie Lentini\n\nA Mexican military helicopter involved in an anti-drug smuggling operation flew into America before shooting at two US Border Patrol agents yesterday.\n\nKVOA-TV reported the chopper fired over Tohono O\xe2\x80\x99odham Indian Nation in the Arizona desert, before flying back to Mexico early Thursday morning. The agents were not injured.\n\nBorder Patrol Tucson Sector union president Art del Cueto said Mexican officials 'contacted US authorities and apologized for the incident' which occurred between midnight and 6am.\n\nLocation: Authorities said the Mexican military helicopter fired two shots over Tohono O'odham Indian Nation in the Arizona desert near the border, before flying back to Mexico early Thursday morning\n\nUS Border Patrol spokesman Andy Adame confirmed the report, saying the 'Mexican law enforcement helicopter' fired two shots 'while on a drug interdiction operation near the border'.\n\nHe said the incid

In [44]:
another_sample_story = tf.io.read_file(another_file_path)
tf.strings.split(another_sample_story, '\n\n')

<tf.Tensor: shape=(19,), dtype=string, numpy=
array([b'By\nRosemarie Lentini',
       b'A Mexican military helicopter involved in an anti-drug smuggling operation flew into America before shooting at two US Border Patrol agents yesterday.',
       b'KVOA-TV reported the chopper fired over Tohono O\xe2\x80\x99odham Indian Nation in the Arizona desert, before flying back to Mexico early Thursday morning. The agents were not injured.',
       b"Border Patrol Tucson Sector union president Art del Cueto said Mexican officials 'contacted US authorities and apologized for the incident' which occurred between midnight and 6am.",
       b"Location: Authorities said the Mexican military helicopter fired two shots over Tohono O'odham Indian Nation in the Arizona desert near the border, before flying back to Mexico early Thursday morning",
       b"US Border Patrol spokesman Andy Adame confirmed the report, saying the 'Mexican law enforcement helicopter' fired two shots 'while on a drug interdicti

In [46]:
another_sample_story_sents = tf.strings.split(another_sample_story, '\n\n')
another_pos = tf.where(another_sample_story_sents=="@highlight")
another_pos = tf.reshape(another_pos, [-1])
another_sample_article = tf.strings.join(another_sample_story_sents[:another_pos[0]], ' ')
another_ind = another_pos + 1
another_highlights = tf.gather(another_sample_story_sents, another_ind)
another_sample_summary = tf.strings.join(another_highlights + '.', ' ')

another_sample_article, another_sample_summary

(<tf.Tensor: shape=(), dtype=string, numpy=b"By\nRosemarie Lentini A Mexican military helicopter involved in an anti-drug smuggling operation flew into America before shooting at two US Border Patrol agents yesterday. KVOA-TV reported the chopper fired over Tohono O\xe2\x80\x99odham Indian Nation in the Arizona desert, before flying back to Mexico early Thursday morning. The agents were not injured. Border Patrol Tucson Sector union president Art del Cueto said Mexican officials 'contacted US authorities and apologized for the incident' which occurred between midnight and 6am. Location: Authorities said the Mexican military helicopter fired two shots over Tohono O'odham Indian Nation in the Arizona desert near the border, before flying back to Mexico early Thursday morning US Border Patrol spokesman Andy Adame confirmed the report, saying the 'Mexican law enforcement helicopter' fired two shots 'while on a drug interdiction operation near the border'. He said the incident was under inv